In [ ]:
from langchain_neo4j import Neo4jGraph
from dotenv import load_dotenv
load_dotenv()

In [6]:
url = "bolt://localhost:7687"
username = "neo4j"
password = "password"

graph = Neo4jGraph(url=url, username=username, password=password)

In [7]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=0,
    model="gpt-4o-mini",
)
llm_transformer = LLMGraphTransformer(llm=llm)

In [8]:
from langchain_community.document_loaders import WikipediaLoader

query = "Warren Buffett"
raw_documents = WikipediaLoader(query=query).load()

In [9]:
graph_documents = llm_transformer.convert_to_graph_documents(raw_documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Warren Edward Buffett', type='Person', properties={}), Node(id='Berkshire Hathaway', type='Company', properties={}), Node(id='Howard Buffett', type='Person', properties={}), Node(id='Bill Gates', type='Person', properties={}), Node(id='Greg Abel', type='Person', properties={}), Node(id='Gates Foundation', type='Organization', properties={}), Node(id='One Thousand Ways To Make $1000', type='Book', properties={}), Node(id='Omaha', type='Location', properties={}), Node(id='Wharton School Of The University Of Pennsylvania', type='School', properties={}), Node(id='University Of Nebraska', type='School', properties={}), Node(id='Columbia Business School', type='School', properties={}), Node(id='New York Institute Of Finance', type='School', properties={}), Node(id='Rose Hill Elementary School', type='School', properties={}), Node(id='Alice Deal Junior High School', type='School', properties={}), Node(id='Woodrow Wilson High School', type='School', properties={})]
Relationship

In [10]:
graph.add_graph_documents(graph_documents, include_source=True)

# Query the graph

In [11]:
from langchain_neo4j import GraphCypherQAChain

# Create the Cypher and QA LLMs

# Cypher LLM for generating Cypher queries
cypher_llm_bedrock_model = ChatOpenAI(
    temperature=0,
    model="gpt-4o",
)

# QA LLM for answering questions based on the Cypher query results
qa_llm_bedrock_model = ChatOpenAI(
    temperature=0,
    model="gpt-4o-mini",
)

chain = GraphCypherQAChain.from_llm(
    cypher_llm=cypher_llm_bedrock_model,
    qa_llm=qa_llm_bedrock_model,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
)

In [14]:
chain.invoke("Who is or was working at Berkshire Hathaway?")



> Entering new GraphCypherQAChain chain...


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The semantics of using colon in the separation of alternative relationship types will change in a future version. (Please use ':WORKS_AT|WORKED_AT' instead)} {position: line: 2, column: 21, offset: 27} for query: 'cypher\nMATCH (p)-[:WORKS_AT|:WORKED_AT]->(c {name: "Berkshire Hathaway"})\nRETURN p\n'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type i

Generated Cypher:
cypher
MATCH (p)-[:WORKS_AT|:WORKED_AT]->(c {name: "Berkshire Hathaway"})
RETURN p

Full Context:
[]

> Finished chain.


{'query': 'Who is or was working at Berkshire Hathaway?',
 'result': "I don't know the answer."}